In [ ]:
import numpy as np
import os
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import accuracy_score
from google.colab import files, drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load the model
model_path = '/content/drive/MyDrive/Models/cnn_rgb_model.h5'
model = load_model(model_path)
print("Model loaded successfully from Drive.")

Model loaded successfully from Drive.


In [ ]:
# load test zip file
print("Upload zip files")
uploaded = files.upload()

Upload zip files


Saving rgb_data.zip to rgb_data.zip


In [ ]:
import zipfile
image_folder = ''
for fname in uploaded.keys():
    # loads test images
    if fname.endswith('.zip'):
      image_folder = os.path.splitext(fname)[0]
      image_folder = os.path.join('/content', image_folder)
      os.makedirs(image_folder, exist_ok=True)
      with zipfile.ZipFile(fname, 'r') as zip_ref:
        zip_ref.extractall('/content')
      print("Images extracted")

Images extracted


In [ ]:
import tensorflow as tf
import pandas as pd
import os

image_csv = 'rgb_image_names.csv'
image_names = pd.read_csv(image_csv, header=None)[0].tolist()
image_paths = [os.path.join(image_folder, name) for name in image_names]
dataset = tf.data.Dataset.from_tensor_slices(image_paths)

def process_image(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.resize(img, [64, 64])
  #img = tf.cast(img, tf.float32) / 255.0
  return img

dataset = dataset.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.batch(32).prefetch(tf.data.AUTOTUNE)
predictions = []

for images in dataset:
  preds = model.predict(images)
  preds = tf.argmax(preds, axis=1)
  predictions.extend(preds.numpy())

predictions_df = pd.DataFrame(predictions, columns=['predictions'])
predictions_df.to_csv('Laksita_rgb.csv', index=False)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
